In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 212.4MB 62kB/s 
     |████████████████████████████████| 204kB 48.2MB/s 


In [45]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
import findspark
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

from pyspark import SparkContext, SparkConf
from pyspark.serializers import MarshalSerializer
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn import metrics
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pyspark.rdd import RDD

def Kmeans(df,k,CT = 0.0001,I = 30,Exp = 10):
  np.random.seed(0)
  #read the data
  dataset = spark.read.csv(df,header = True, inferSchema=True)
  dataset = np.array(dataset.collect())
  #clean the data
  target_class = dataset[:,len(dataset[0])-1:]
  dataset = dataset[:,:len(dataset[0])-1]
  #normalize the data
  min_max_scaler = MinMaxScaler()
  dataset = min_max_scaler.fit_transform(dataset)
  #Spark Connection  
  sc = SparkContext.getOrCreate();
  list_ch = []
  list_ari = []
  num = 316533652
  for i in range(Exp):
    np.random.seed(num)
    num +=1
    #k random smaple from the data
    number_of_rows = dataset.shape[0]
    random_indices = np.random.choice(number_of_rows, size=k, replace=False)
    random_rows = dataset[random_indices, :]
    random_rows = [list(i) for i in random_rows]
    round = 0 
    #Iterations
    while(True):
      round += 1
      # Map function for any point that setting nearest centroid
      rdd = sc.parallelize(dataset).map(lambda x: (mapping(x,random_rows), x))
      # Reduce function calculate new centroid
      redu = rdd.groupByKey().mapValues(lambda x: sum(x)/len(x)).collect()
      new_random_rows = [list(i[1]) for i in redu]

      # Check iteration number
      if (round >= I):
        break
      # Check CT
      if all([sum([(random_rows[i][j] - new_random_rows[i][j])**2 for j in range(len(new_random_rows[0]))])**0.5 < CT for i in range(len(new_random_rows))]):
        break
      random_rows = new_random_rows
    
    # orgenize data for evaluate the algorithem 
    labels = [k for k,v in rdd.collect()]
    X = [v for k,v in rdd.collect()]
    target_classnew = []
    for i in target_class:
      target_classnew.append(int(i[0]))
    # calinski_harabasz_score test
    list_ch.append(metrics.calinski_harabasz_score(X, labels))
    # adjusted_rand_score test
    list_ari.append(metrics.adjusted_rand_score(target_classnew, labels))
  print((float(np.average(np.array(list_ch))),float(np.std(np.array(list_ch)))))
  print((float(np.average(np.array(list_ari))),float(np.std(np.array(list_ari)))))


def mapping(point,samples):
  dist_lst = []
  for sample in samples:
    num = 0
    for p in range(len(sample)):
        # calculate distance
        num += (point[p] - sample[p])**2
    dist_lst.append((num**0.5,sample))
  sorted_dist = sorted(dist_lst, key=lambda tup: tup[0])
  # take the minimum distance and classify to group number
  val = samples.index(sorted_dist[0][1])
  return val
#
# exprianses = ["/content/iris.csv", "/content/glass.csv", "/content/parkinsons.csv"]
# for i in exprianses:
#   for j in range(5):
#       Kmeans(i,3+j)

(339.6770867425542, 45.506302842779434)
(0.6814128473033063, 0.0844695226515022)
(301.9862603341797, 16.58275030568852)
(0.6104465255769392, 0.021970427328270932)
(274.45823131694476, 15.198384303298585)
(0.5571147294277533, 0.08630562896274031)
(242.19269263446873, 17.72136265234646)
(0.5268871016158485, 0.09084112043222263)
(227.51891363963523, 14.238945866751704)
(0.474659000961955, 0.07837420833746389)
(100.27206051165317, 6.072329316605357)
(0.17922411429145174, 0.05148410030751475)
(88.98741655603496, 8.525298857029004)
(0.15587057875097673, 0.04396039347356883)
(76.1751597476502, 10.052657652038631)
(0.177125407558047, 0.037884460982762685)
(74.0453155215632, 6.538548542071104)
(0.16941330287605103, 0.032783262641878114)


KeyboardInterrupt: ignored